# Running toxicity on The Tyee comments

In [1]:
import pandas as pd
import os
import csv
import json
import sys
import random
import requests
import time
from bs4 import BeautifulSoup

In [2]:
from googleapiclient import discovery
from googleapiclient import http
API_KEY='REPLACE_WITH_YOUR_API_KEY'

In [3]:
service = discovery.build('commentanalyzer', 'v1alpha1', developerKey=API_KEY)
numRequests = 0

def getPrediction(text):
    global numRequests
    global startTime
    
    if len(text) > 20480:
        return -1
    
    analyze_request = {
      'comment': { 'text': text },
      'languages' : ["en"],
      'requestedAttributes': {'TOXICITY': {}}
    }
    
    # 10 requests per second
    
    if numRequests >= 990:
        endTime = time.time()
        while (endTime - startTime) <= 100:
            time.sleep(1)
            endTime = time.time()
        numRequests = 0
        startTime = time.time()

    rateLimitExceeded = True
    while rateLimitExceeded:
        try:
            response = service.comments().analyze(body=analyze_request).execute()
        except http.HttpError:
            time.sleep(5)
        else:
            rateLimitExceeded = False
        finally:
            numRequests += 1

    return response['attributeScores']['TOXICITY']['summaryScore']['value']

## Testing

In [4]:
text = 'I hope your mother gets cervical cancer.'

In [5]:
getPrediction(text)

0.9471205

## Reading in Tyee data

In [6]:
raw = pd.read_csv('/home/vgautam/the_tyee_posts_preprocessed.csv')['message_preprocessed'].dropna()

df = pd.DataFrame(raw.rename('pp_comment_text'))
df = df[df['pp_comment_text'] != '']

/home/vgautam/anaconda3/envs/usra/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df.head()

,pp_comment_text
0,"Tyee Site Manager Hello Everyone, The Tyee ex..."
1,It has come to the Tyee editor's attention tha...
2,"I'm glad you're tackling this ""alienation"" wor..."
3,Very charming and descriptive. I enjoyed this ...
4,Looking forward to seeing some stories of posi...


## Predicting toxicity

In [ ]:
startTime = time.time()
with open('/home/vgautam/new_tyee_toxicity_results.csv', 'w+') as fo:
    fieldnames = ['pp_comment_text', 'TOXICITY_probability']
    
    writer = csv.DictWriter(fo, fieldnames=fieldnames)
    
    writer.writeheader()

    for i,post in enumerate(list(df['pp_comment_text'])):
        if (i % 100000) == 0: print(i)
        tox_dict = {}
        
        tox_dict['pp_comment_text'] = post
        tox_dict['TOXICITY_probability'] = getPrediction(post)
        
        writer.writerow(tox_dict)

0


## Preview of results

In [ ]:
pd.read_csv('/home/vgautam/new_tyee_toxicity_results.csv').head()